### The STV-FIS effort is funded by the NASA Decadal Survey Incubation program - **NNH21ZDA001N-DSI**

# SDAP Tomogram & LIDAR Subsetting & Visualization Endpoints

This notebook showcases some vizualization capabilities built directly into SDAP for the purposes of visuzlizing SAR, LIDAR, and ML-Fusion data. The following demo products were used:
- SAR: Work in progress geolocated product (not publicly availabe). Kings Canyon, CA; Lope Nat'l Park, Gabon (HH, HV, VV); Rabi Forest, Gabon (HH, HV, VV); Redwood Nat'l Park, CA (HH, HV, VV | L- & P-band)
- LIDAR: [This dataset](https://daac.ornl.gov/ABOVE/guides/ABoVE_LVIS_VegetationStructure.html) (ABoVE: LVIS L3 Gridded Vegetation Structure across North America, 2017 and 2019)
- Kauri: An ML fusion product using SAR and LIDAR data to predict RH98 (canopy height).

In [1]:
import json
import os
import sys
from base64 import b64encode
from copy import deepcopy
from datetime import datetime
from io import BytesIO, StringIO
from math import floor, ceil
from zipfile import ZipFile

import fiona
import ipywidgets as widgets
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import urllib3
import xarray as xr
from ipyleaflet import DrawControl, Map, basemaps, GeoData, ImageOverlay
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from requests.compat import urlencode
from shapely import from_geojson
from shapely.geometry import Point
from IPython.display import display
from IPython.display import Image as IpImage
from PIL import Image

fiona.drvsupport.supported_drivers["LIBKML"] = "rw"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# NOTE: Disabling verification due to certificate issues on my local machine

In [2]:
BASE_URL = 'https://ideas-digitaltwin.jpl.nasa.gov'
NEXUS_URL = f'{BASE_URL}/nexus'

DATASETS_URL = f'{NEXUS_URL}/list'

TOMOGRAM_LAT_URL = f'{NEXUS_URL}/tomogram/latitude'
TOMOGRAM_LON_URL = f'{NEXUS_URL}/tomogram/longitude'
TOMOGRAM_CUS_URL = f'{NEXUS_URL}/tomogram/profile'
TOMOGRAM_3D_URL = f'{NEXUS_URL}/tomogram/3d'

LIDAR_URL = f'{NEXUS_URL}/stv/lidar'

TOMOGRAM_DS_KC = 'Kings_Canyon_Tomogram'
TOMOGRAM_DS_Lope = 'Lope_Tomogram_{}'
TOMOGRAM_DS_Rabi = 'Rabi_Tomogram_{}'
TOMOGRAM_DS_Redwood_Lband = 'Redwood_Tomogram_L_{}'
TOMOGRAM_DS_Redwood_Pband = 'Redwood_Tomogram_P_{}'

LIDAR_DS = 'LVIS_ABoVE'

KAURI_SAMPLE_DS = 'Kauri_sample_output'

TOMO_MAP_DATASETS = { 
    TOMOGRAM_DS_Rabi.format('HH'): (
        [
            'Gabon_LVIS_DSM', 
            'Rabi_HybridSAR_L_H5_AfriSAR_DSM', 
            'rabi_L_KauriCNN_canopy_elevation_5x5_200epochs', 
        ], 
        [
            'Gabon_LVIS_ZG', 
            'rabi_L_KauriCNN_ground_elevation_dem_5x5_60epochs', 
        ]
    ),
    TOMOGRAM_DS_Rabi.format('HV'): (
        [
            'Gabon_LVIS_DSM', 
            'Rabi_HybridSAR_L_H5_AfriSAR_DSM', 
            'rabi_L_KauriCNN_canopy_elevation_5x5_200epochs', 
        ], 
        [
            'Gabon_LVIS_ZG', 
            'rabi_L_KauriCNN_ground_elevation_dem_5x5_60epochs', 
        ]
    ),
    TOMOGRAM_DS_Rabi.format('VV'): (
        [
            'Gabon_LVIS_DSM', 
            'Rabi_HybridSAR_L_H5_AfriSAR_DSM', 
            'rabi_L_KauriCNN_canopy_elevation_5x5_200epochs', 
        ], 
        [
            'Gabon_LVIS_ZG', 
            'rabi_L_KauriCNN_ground_elevation_dem_5x5_60epochs', 
        ]
    ),
    TOMOGRAM_DS_Lope.format('HH'): (
        ['Gabon_LVIS_DSM', 'Lope_HybridSAR_L_H5_AfriSAR_DSM'], 
        ['Gabon_LVIS_ZG', 'Lope_Reference_DTM']
    ),
    TOMOGRAM_DS_Lope.format('HV'): (
        ['Gabon_LVIS_DSM', 'Lope_HybridSAR_L_H5_AfriSAR_DSM'], 
        ['Gabon_LVIS_ZG', 'Lope_Reference_DTM']
    ),
    TOMOGRAM_DS_Lope.format('VV'): (
        ['Gabon_LVIS_DSM', 'Lope_HybridSAR_L_H5_AfriSAR_DSM'], 
        ['Gabon_LVIS_ZG', 'Lope_Reference_DTM']
    ),
    (TOMOGRAM_DS_Rabi + '_NO_DEM').format('HH'): (
        [
            'Gabon_LVIS_RH098', 
            'Rabi_HybridSAR_L_H5_AfriSAR_CHM', 
            'rabi_L_KauriCNN_canopy_height_5x5_200epochs'
        ], 
        [
            'rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs'
        ]),
    (TOMOGRAM_DS_Rabi + '_NO_DEM').format('HV'): (
        [
            'Gabon_LVIS_RH098', 
            'Rabi_HybridSAR_L_H5_AfriSAR_CHM', 
            'rabi_L_KauriCNN_canopy_height_5x5_200epochs'
        ], 
        [
            'rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs']),
    (TOMOGRAM_DS_Rabi + '_NO_DEM').format('VV'): (
        [
            'Gabon_LVIS_RH098', 
            'Rabi_HybridSAR_L_H5_AfriSAR_CHM', 
            'rabi_L_KauriCNN_canopy_height_5x5_200epochs'
        ], 
        [
            'rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs'
        ]),
    (TOMOGRAM_DS_Lope + '_NO_DEM').format('HH'): (
        [
            'Gabon_LVIS_RH098', 
            'Lope_HybridSAR_L_H5_AfriSAR_CHM' 
        ], 
        []
    ),
    (TOMOGRAM_DS_Lope + '_NO_DEM').format('HV'): (
        [
            'Gabon_LVIS_RH098', 
            'Lope_HybridSAR_L_H5_AfriSAR_CHM'
        ], 
        []
    ),
    (TOMOGRAM_DS_Lope + '_NO_DEM').format('VV'): (
        [
            'Gabon_LVIS_RH098', 
            'Lope_HybridSAR_L_H5_AfriSAR_CHM'
        ], 
        []
    ),
}


def get(url, params=None):
    t = datetime.now()
        
    print(f'{url}', end='')

    if params is not None:
        print(f'?{urlencode(params)}')
    else:
        print()
        params = dict()
    
    print('Waiting for response from SDAP...', end='')
    response = requests.get(url, params=params, verify=False)
    td = datetime.now() - t

    if response.status_code == 200:
        print(f' Done in {td}')
        return response
    elif response.status_code == 200:
        print(f' Timed out after {td}')
        return None
    else:
        print(f' Failed in {td}')
        print(response.text)
        response.raise_for_status()

In [3]:
# Audit available datasets

datasets = []
tomo_datasets = [TOMOGRAM_DS_KC]

for tomo_site in [TOMOGRAM_DS_Lope, TOMOGRAM_DS_Rabi, TOMOGRAM_DS_Redwood_Lband, TOMOGRAM_DS_Redwood_Pband]:
    for pol in ['HH', 'HV', 'VV']:
        tomo_datasets.extend([tomo_site.format(pol), tomo_site.format(pol) + "_NO_DEM"])

datasets.extend(tomo_datasets)

datasets.append(LIDAR_DS)

maps = []

for v in TOMO_MAP_DATASETS.values():
    for p in v:
        maps.extend(p)

datasets.extend(set(maps))

datasets.append(KAURI_SAMPLE_DS)

datasets.sort()

print(f'Found {len(datasets)} datasets described in this notebook')

available_datasets = [ds['shortName'] for ds in get(DATASETS_URL).json()]

print(f'Found {len(available_datasets)} datasets available in {BASE_URL}')

missing = list(set(datasets).difference(set(available_datasets)))

if len(missing) > 0:
    print(f'WARNING: Found {len(missing)} datasets used in this notebook that are missing from {BASE_URL}', file=sys.stderr)
    for m in sorted(missing):
        print(f'\t- {m}', file=sys.stderr)
    print(file=sys.stderr)
    print('Make note of these and avoid using queries that reference them', file=sys.stderr)
else:
    print('All datasets are present')

Found 39 datasets described in this notebook
https://ideas-digitaltwin.jpl.nasa.gov/nexus/list
Waiting for response from SDAP... Done in 0:00:00.200912
Found 334 datasets available in https://ideas-digitaltwin.jpl.nasa.gov


	- LVIS_ABoVE
	- Lope_Tomogram_HH
	- Lope_Tomogram_HH_NO_DEM
	- Lope_Tomogram_HV
	- Lope_Tomogram_HV_NO_DEM
	- Lope_Tomogram_VV
	- Lope_Tomogram_VV_NO_DEM
	- Redwood_Tomogram_L_HH
	- Redwood_Tomogram_L_HH_NO_DEM
	- Redwood_Tomogram_L_HV
	- Redwood_Tomogram_L_HV_NO_DEM
	- Redwood_Tomogram_L_VV
	- Redwood_Tomogram_L_VV_NO_DEM
	- Redwood_Tomogram_P_HH
	- Redwood_Tomogram_P_HH_NO_DEM
	- Redwood_Tomogram_P_HV
	- Redwood_Tomogram_P_HV_NO_DEM
	- Redwood_Tomogram_P_VV
	- Redwood_Tomogram_P_VV_NO_DEM

Make note of these and avoid using queries that reference them


## Tomograms

### Tomogram Profile Plots

In [ ]:
sites = [
    ('Kings Canyon National Park, US', 'kcnp'),
    ('Lope National Park, GA', 'lope'),
    ('Rabi Forest, GA', 'rabi'),
    ('Redwood National Park, US (L-band)', 'redl'),
    ('Redwood National Park, US (P-band)', 'redp'),
]

site_sel = widgets.Dropdown(options=sites, description='Site: ', disabled=False, layout = widgets.Layout(width='350px'))
pol_sel = widgets.Dropdown(options=['HH', 'HV', 'VV'], description='Pol: ', disabled=False, layout = widgets.Layout(width='350px'))
dim_sel = widgets.Dropdown(options=['Latitude', 'Longitude', 'Custom'], description='Direction: ', disabled=False, layout = widgets.Layout(width='350px'))
dem_sel = widgets.Checkbox(value=True, disabled=False, description='Add DEM')
peaks = widgets.Checkbox(value=False, disabled=False, description='Show peaks in plot')


display(site_sel)
display(pol_sel)
display(dim_sel)
display(dem_sel)
display(peaks)

In [ ]:
PARAMS = {
    'kcnp': dict(ds=TOMOGRAM_DS_KC,            bounds=dict(min_lon=-119.60,  max_lon=-118.75,  min_lat=36.98,  max_lat=37.15,  min_elev=66.84,  max_elev=3732.20, min_elev_nd=-60,  max_elev_nd=60), kmzpng='kmz/kingsF_09433_22046_021_220929_PL09043020_05_CX_01'),
    'lope': dict(ds=TOMOGRAM_DS_Lope,          bounds=dict(min_lon=11.43,    max_lon=11.83,    min_lat=-0.34,  max_lat=0.07,   min_elev=2.5,    max_elev=717.87,  min_elev_nd=-102, max_elev_nd=56), kmzpng='kmz/lopenp_14043_16008_009_160225_L090_CX_129A_02'),
    'rabi': dict(ds=TOMOGRAM_DS_Rabi,          bounds=dict(min_lon=9.70,     max_lon=10.11,    min_lat=-2.05,  max_lat=-1.65,  min_elev=-94.90, max_elev=236.65,  min_elev_nd=-102, max_elev_nd=56), kmzpng='kmz/rabifo_13049_16010_010_160228_L090_CX_129A_03'),
    'redl': dict(ds=TOMOGRAM_DS_Redwood_Lband, bounds=dict(min_lon=-124.247, max_lon=-123.903, min_lat=41.571, max_lat=41.888, min_elev=-89.13, max_elev=1147.06, min_elev_nd=-60,  max_elev_nd=60), kmzpng='kmz/redwoo_16912_22043_019_220913_L090_CX_01'),
    'redp': dict(ds=TOMOGRAM_DS_Redwood_Pband, bounds=dict(min_lon=-124.247, max_lon=-123.896, min_lat=41.528, max_lat=41.888, min_elev=-89.22, max_elev=1146.80, min_elev_nd=-60,  max_elev_nd=60), kmzpng='kmz/redwoo_16912_22043_019_220913_L090_CX_01'),
}

params = PARAMS[site_sel.value]

if site_sel.value != 'kcnp':
    params['ds'] = params['ds'].format(pol_sel.value)
    
if not dem_sel.value:
    params['ds'] = params['ds'] + '_NO_DEM'
    params['bounds']['min_elev'] = params['bounds']['min_elev_nd']
    params['bounds']['max_elev'] = params['bounds']['max_elev_nd']

del params['bounds']['min_elev_nd']
del params['bounds']['max_elev_nd']

line = {
    'start-point': 'invalid',
    'end-point': 'invalid'
}

if dim_sel.value in ['Longitude', 'Latitude']:
    s = widgets.FloatSlider(
        value=((params['bounds']['min_lon'] + params['bounds']['max_lon']) / 2) if dim_sel.value == 'Longitude' else ((params['bounds']['min_lat'] + params['bounds']['max_lat']) / 2),
        min=params['bounds']['min_lon'] if dim_sel.value == 'Longitude' else params['bounds']['min_lat'],
        max=params['bounds']['max_lon'] if dim_sel.value == 'Longitude' else params['bounds']['max_lat'],
        step=0.001,
        description=f'{dim_sel.value}: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.3f',
        layout = widgets.Layout(width='500px')
    )
    
    d = widgets.FloatRangeSlider(
        value=[
            params['bounds']['min_lat'] if dim_sel.value == 'Longitude' else params['bounds']['min_lon'],
            params['bounds']['max_lat'] if dim_sel.value == 'Longitude' else params['bounds']['max_lon']
        ],
        min=params['bounds']['min_lat'] if dim_sel.value == 'Longitude' else params['bounds']['min_lon'],
        max=params['bounds']['max_lat'] if dim_sel.value == 'Longitude' else params['bounds']['max_lon'],
        step=0.01,
        description='Latitude: ' if dim_sel.value == 'Longitude' else 'Longitude: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(width='500px')
    )
    
    display(s)
    display(d)
else:
    kmz_png = params['kmzpng']

    gdf = gpd.read_file(kmz_png + '.kmz')
    center = gdf.to_crs('EPSG:4326').dissolve().centroid.get_coordinates().stack()[::-1].tolist()

    m = Map(center = center, zoom = 10, min_zoom = 9, max_zoom = 20, basemap=basemaps.Esri.WorldImagery)

    draw_control = DrawControl()

    draw_control.circlemarker = {}
    draw_control.polygon = {}
    
    line = {
        'start-point': 'invalid',
        'end-point': 'invalid'
    }
    
    def handle_draw(self, action, geo_json):
        try:
            geom = from_geojson(json.dumps(geo_json))
            points = list(geom.coords)
        
            if len(points) == 2:
                line['start-point'] = Point(points[0])    
                line['end-point'] = Point(points[1])
            else:
                line['start-point'] = 'invalid'
                line['end-point'] = 'invalid'
        except Exception as err:
            line['start-point'] = 'invalid'
            line['end-point'] = 'invalid'
            line['err'] = repr(err)
    
    draw_control.on_draw(handle_draw)
    
    m.add_control(draw_control)
    
    gdata = GeoData(geo_dataframe=gdf, style={'fillOpacity': 0})
    
    bounds = gdf.dissolve().to_crs('EPSG:4326').bounds.iloc[0]
    bounds = ((bounds.miny, bounds.minx), (bounds.maxy, bounds.maxx))
    
    image = Image.open(kmz_png + '.png')
    buf = BytesIO()
    
    ext = 'png'
    
    image.save(buf, ext)
    
    img_data = b64encode(buf.getvalue())
    img_data = img_data.decode('ascii')
    url = 'data:image{};base64,'.format(ext) + img_data
    
    img = ImageOverlay(url=url, bounds=bounds)
    
    m.add(gdata)
    m.add(img)
    
    display(m)

e = widgets.IntRangeSlider(
    value=[params['bounds']['min_elev'], params['bounds']['max_elev']],
    min=floor(params['bounds']['min_elev']),
    max=ceil(params['bounds']['max_elev']),
    step=1,
    description='Elevation: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout = widgets.Layout(width='500px')
)

display(e)
    
options_c, options_g = deepcopy(TOMO_MAP_DATASETS).get(params['ds'], ([], []))

if '--' not in options_c:
    options_c.append('--')

if '--' not in options_g:
    options_g.append('--')

c_sel = widgets.Dropdown(options=options_c, description='Primary Canopy height map: ', value='--', disabled=False, layout = widgets.Layout(width='500px'), style={'description_width': 'initial'})
g_sel = widgets.Dropdown(options=options_g, description='Primary Ground height map: ', value='--', disabled=False, layout = widgets.Layout(width='500px'), style={'description_width': 'initial'})

c2_sel = widgets.Dropdown(options=options_c, description='Secondary Canopy height map: ', value='--', disabled=False, layout = widgets.Layout(width='500px'), style={'description_width': 'initial'})
g2_sel = widgets.Dropdown(options=options_g, description='Secondary Ground height map: ', value='--', disabled=False, layout = widgets.Layout(width='500px'), style={'description_width': 'initial'})

display(c_sel)
display(c2_sel)
display(g_sel)
display(g2_sel)

percentile = widgets.Checkbox(value=False, disabled=False, description='Render by cumulative vertical return power', style={'description_width': 'initial'})
display(percentile)

In [ ]:
if dim_sel.value in ['Longitude', 'Latitude']:
    if dim_sel.value == 'Longitude':
        params['longitude'] = s.value
        (params['minLat'], params['maxLat']) = d.value
        url = TOMOGRAM_LON_URL
    else:
        params['latitude'] = s.value
        (params['minLon'], params['maxLon']) = d.value
        url = TOMOGRAM_LAT_URL
else:
    draw_control.clear_polylines()
    if not isinstance(line['start-point'], Point):
        raise ValueError('Invalid line selection')

    start = line['start-point']
    end = line['end-point']
    
    params['p1'] = f'{start.x},{start.y}'
    params['p2'] = f'{end.x},{end.y}'
    
    url = TOMOGRAM_CUS_URL

(params['minElevation'], params['maxElevation']) = e.value
params['output'] = 'PNG'
params['peaks'] = peaks.value

if c_sel.value == '--' and c2_sel.value != '--':
    raise ValueError('Must specify primary to specify secondary')

if g_sel.value == '--' and g2_sel.value != '--':
    raise ValueError('Must specify primary to specify secondary')

if c_sel.value != '--':
    params['canopy_ds'] = c_sel.value
    if c2_sel.value != '--':
        params['canopy_ds'] = params['canopy_ds'] + ',' + c2_sel.value

if g_sel.value != '--':
    params['ground_ds'] = g_sel.value
    if g2_sel.value != '--':
        params['ground_ds'] = params['ground_ds'] + ',' + g2_sel.value

params['elevPercentiles'] = percentile.value

response = get(url, {p: params[p] for p in params if p not in ['bounds', 'kmzpng']})

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

## Tomogram Profile Plots - Fixed Examples

### Tomograms - Slicing along single lats/lons

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_KC + '_NO_DEM',
    output='PNG',
    latitude=37.055,
    minLon=-119.24,
    maxLon=-119.2,
    minElevation=-121,
    maxElevation=121,
    peaks=True
)

response = get(TOMOGRAM_LAT_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_KC,
    output='PNG',
    longitude=-119.2103533733451,
    minLat=37,
    maxLat=37.25,
    minElevation=1000,
    maxElevation=2000,
    peaks=False
)

response = get(TOMOGRAM_LON_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Lope.format('VV'),
    output='PNG',
    longitude=11.6,
    minLat=-0.075,
    maxLat=-0.05,
    minElevation=5,
    maxElevation=350,
    peaks=False
)

response = get(TOMOGRAM_LON_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HV'),
    output='PNG',
    latitude=-1.85,
    minLon=9.8,
    maxLon=10,
    minElevation=-90,
    maxElevation=1200,
    peaks=False
)

response = get(TOMOGRAM_LAT_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HV') + '_NO_DEM',
    output='PNG',
    longitude=9.98,
    minLat=-1.972,
    maxLat=-1.935,
    minElevation=-40,
    maxElevation=120,
    peaks=False,
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
    elevPercentiles=True
)

response = get(TOMOGRAM_LON_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HV') + '_NO_DEM',
    output='PNG',
    longitude=9.98,
    minLat=-1.972,
    maxLat=-1.935,
    minElevation=-40,
    maxElevation=120,
    peaks=False,
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
    elevPercentiles=True,
    cmap='forest'
)

response = get(TOMOGRAM_LON_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

### Tomograms - Slicing along range
<!-- is there a better way to title this section??? /-->


In [ ]:
params = dict(
    ds=TOMOGRAM_DS_KC,
    output='GIF',
    longitude='-119.2103533733451:-119.2082533733451',
    minLat=37,
    maxLat=37.25,
    minElevation=1000,
    maxElevation=2000,
    peaks=True,
)

response = get(TOMOGRAM_LON_URL, params)

img = IpImage(data=response.content, format='gif')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HV'),
    output='GIF',
    longitude='9.97:9.99',
    minLat=-1.97,
    maxLat=-1.93,
    minElevation=10,
    maxElevation=150,
    peaks=False,
    canopy_ds='rabi_L_KauriCNN_canopy_elevation_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_dem_5x5_60epochs'
)

response = get(TOMOGRAM_LON_URL, params)

img = IpImage(data=response.content, format='gif')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HV'),
    output='GIF',
    longitude='9.97:9.99',
    minLat=-1.97,
    maxLat=-1.93,
    minElevation=10,
    maxElevation=150,
    peaks=False,
    canopy_ds='rabi_L_KauriCNN_canopy_elevation_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_dem_5x5_60epochs',
    elevPercentiles=True
)

response = get(TOMOGRAM_LON_URL, params)

img = IpImage(data=response.content, format='gif')
display(img)

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('HH'),
    output='GIF',
    latitude='-1.86:-1.84',
    minLon=9.8,
    maxLon=9.935,
    minElevation=-90,
    maxElevation=1200,
    peaks=False,
)

response = get(TOMOGRAM_LAT_URL, params)

img = IpImage(data=response.content, format='gif')
display(img)

### Tomograms - 3D Plot Examples

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV'),
    output='PNG',
    b='9.85,-1.87,9.86,-1.86',
    minElevation=-95,
    maxElevation=240,
    viewAngle='225,15'
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

#### 3D Masking & Rendering options

In [ ]:
# Raw tomogram

params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV') + '_NO_DEM',
    output='PNG',
    b='9.95,-1.943,10,-1.893',
    minElevation=-102,
    maxElevation=56,
    viewAngle='225,15'
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
# Masked tomogram

params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV') + '_NO_DEM',
    output='PNG',
    b='9.95,-1.943,10,-1.893',
    minElevation=-40,
    maxElevation=56,
    viewAngle='225,30',
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
# Percentile rendering

params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV') + '_NO_DEM',
    output='PNG',
    b='9.95,-1.943,10,-1.893',
    minElevation=-40,
    maxElevation=56,
    viewAngle='225,30',
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
    elevPercentiles=True,
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
# RH98 rendering w/ custom color map

params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV') + '_NO_DEM',
    output='PNG',
    b='9.95,-1.943,10,-1.893',
    minElevation=-40,
    maxElevation=56,
    viewAngle='225,30',
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
    elevPercentiles=True,
    renderRH=98,
    cmap='forest',
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

In [ ]:
# RH98 rendering w/ custom color map

params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV') + '_NO_DEM',
    output='PNG',
    b='9.97,-1.943,9.99,-1.923',
    minElevation=-40,
    maxElevation=56,
    viewAngle='225,30',
    canopy_ds='rabi_L_KauriCNN_canopy_height_5x5_200epochs',
    ground_ds='rabi_L_KauriCNN_ground_elevation_bias_5x5_60epochs',
    elevPercentiles=True,
    renderRH=98,
    cmap='forest',
)

response = get(TOMOGRAM_3D_URL, params)

buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

### Tomograms - Other Output Formats

Tomograms subsets can be returned in a simple CSV format to be used with VR tools to enable users to interactively manipulate and explore the data.

In [ ]:
params = dict(
    ds=TOMOGRAM_DS_Rabi.format('VV'),
    output='CSV',
    b='9.95,-1.97,10,-1.92',
    minElevation=-95,
    maxElevation=240
)

response = get(TOMOGRAM_3D_URL, params)

buf = StringIO(response.text)

df = pd.read_csv(buf, names=['x', 'y', 'z', 'value'])
df

In [ ]:
df.to_csv('tomo.ply', header=False, index=False)

## LIDAR

The following cells will showcase SDAP's LIDAR endpoint (Still a WIP)

### LIDAR - PNG Output

In [ ]:
params = dict(
    ds=LIDAR_DS,
    b='-148.8,66.025,-148.5,66.18',
    startTime='2016-06-28T00:00:00Z',
    endTime='2020-06-30T00:00:00Z',
    output='PNG',
    mapToGrid=True
)

response = get(LIDAR_URL, params)
response.raise_for_status()
buf = BytesIO(response.content)

img = Image.open(buf).convert('RGB')
display(img)

### LIDAR - NetCDF Output & Profile Slicing

Profile slicing can be defined by longitude, latitude, or an arbitrary WKT LineString.

It is supported by all output formats except CSV, but showcased here for the NetCDF output format as the plotting in the PNG format is still an awkward work in progress.

In [ ]:
params = dict(
    ds=LIDAR_DS,
    b='-122.3,46,-122,46.25',
    startTime='2016-06-28T00:00:00Z',
    endTime='2020-06-30T00:00:00Z',
    output='NetCDF',
    filename='lidar.nc',
    mapToGrid=True,
    sliceSamples=10000,
    sliceWKT='LINESTRING (-122.12 46.07, -122.09095182261792 46.0220916078604)',
    latSlice=46.09
)

response = get(LIDAR_URL, params)
response.raise_for_status()
buf = BytesIO(response.content)

ds = xr.open_dataset(buf).isel(time=0)
ds

In [ ]:
ds.ground_height.plot(cmap='viridis'); plt.show()
ds.mean_veg_height.plot(cmap='viridis', vmin=0); plt.show()
ds.canopy_height.plot(cmap='viridis', vmin=0, vmax=60); plt.show()
ds.canopy_coverage.plot(cmap='viridis', vmin=0, vmax=100); plt.show()

In [ ]:
x_pts = ds.lon.to_numpy()
rh50_pts = ds['lat_slice_mean_veg_height'].to_numpy()
rh98_pts = ds['lat_slice_canopy_height'].to_numpy()
zg_pts = ds['lat_slice_ground_height'].to_numpy()
cc_pts = ds['lat_slice_canopy_coverage'].to_numpy()

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10, 8))

ax1.plot(
    x_pts, zg_pts,
    x_pts, zg_pts + rh50_pts,
    x_pts, zg_pts + rh98_pts,
)
ax1.set_title('Longitudunal profile heights (m)')
ax1.legend([
    'Ground Height',
    'Mean Vegetation Height',
    'Canopy Height'
])
ax1.set_ylabel('Height [m]')

ax2.plot(x_pts, cc_pts)
ax2.set_title('Longitudunal profile canopy coverage (%)')
ax2.legend(['Canopy Coverage'])
ax2.set_ylim([0, 100])
ax2.set_xlabel('Longitude [deg]')
ax2.set_ylabel('Height [m]')

plt.show()

In [ ]:
x_pts = ds.distance.to_numpy()
rh50_pts = ds['slice_mean_veg_height'].to_numpy()
rh98_pts = ds['slice_canopy_height'].to_numpy()
zg_pts = ds['slice_ground_height'].to_numpy()
cc_pts = ds['slice_canopy_coverage'].to_numpy()

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10, 8))

ax1.plot(
    x_pts, zg_pts,
    x_pts, zg_pts + rh50_pts,
    x_pts, zg_pts + rh98_pts,
)
ax1.set_title('Custom slice heights (m)')
ax1.legend([
    'Ground Height',
    'Mean Vegetation Height',
    'Canopy Height'
])
ax1.set_ylabel('Height [m]')

ax2.plot(x_pts, cc_pts)
ax2.set_title('Custom slice canopy coverage (%)')
ax2.legend(['Canopy Coverage'])
ax2.set_ylim([0, 100])
ax2.set_ylabel('Height [m]')
ax2.set_xlabel('Distance along line [m]')

plt.show()

### LIDAR - CSV & ZIP Outputs

In [ ]:
params = dict(
    ds=LIDAR_DS,
    b='-122.3,46,-122,46.25',
    startTime='2016-06-28T00:00:00Z',
    endTime='2020-06-30T00:00:00Z',
    output='CSV',
    filename='lidar.csv',
    mapToGrid=True,
)

response = get(LIDAR_URL, params)
response.raise_for_status()
buf = StringIO(response.text)

df = pd.read_csv(buf)
df

In [ ]:
# .ply formatting

df[['longitude', 'latitude', 'canopy_height', 'canopy_coverage']].to_csv('lidar.ply', header=False, index=False)

In [ ]:
params = dict(
    ds=LIDAR_DS,
    b='-122.3,46,-122,46.25',
    startTime='2016-06-28T00:00:00Z',
    endTime='2020-06-30T00:00:00Z',
    output='ZIP',
    filename='lidar.zip',
    mapToGrid=True,
    sliceSamples=10000,
    sliceWKT='LINESTRING (-122.12 46.07, -122.09095182261792 46.0220916078604)',
    latSlice='46.09,-122.15,-122.11'
)

response = get(LIDAR_URL, params)
response.raise_for_status()
buf = BytesIO(response.content)

zipfile = ZipFile(buf)
zipfile.namelist()

In [ ]:
for csv in zipfile.namelist():
    print(f'{csv}:')
    csv_buf = StringIO(zipfile.read(csv).decode('utf-8'))
    csv_data = pd.read_csv(csv_buf)
    display(csv_data)

## Kauri output

In [ ]:
n_epochs_map = {  # Map number of training epochs used for the model to the timestamp of the data product it created
    20: '2024-04-16T11:12:00Z',
    60: '2024-05-07T17:54:36Z',
    200: '2024-05-07T22:54:05Z' if not LOCAL else '2024-05-07T17:54:36Z'
}

bbox_map = dict(
    full='9.70,-2.05,10.11,-1.65',
    closeup='9.95,-2.0,10.05,-1.9'
)

output_map = {}

def kauri(b, n_epochs, ax):
    if n_epochs not in n_epochs_map:
        raise ValueError(f'No associated timestamp for {n_epochs=}')

    timestamp = n_epochs_map[n_epochs]
    
    params = {
        'dataset' : KAURI_SAMPLE_DS,
        'b' : bbox_map[b],
        'startTime' : timestamp, 
        'endTime' : timestamp,
        'output' : 'ZIP'
    }
    
    url = f'{NEXUS_URL}/cdmssubset'
    
    t = datetime.now()
        
    try:
        print('Waiting for response from SDAP...', end='')
        response = requests.get(url, params=params, verify=False)
        td = datetime.now() - t
        response.raise_for_status()
    
        print(f' Done in {td}')
    except:
        td = datetime.now() - t
        print(f' Failed in {td}')
        print(response.text)
        raise
        
    response = BytesIO(response.content)
    
    print('Unpacking results')
    
    with ZipFile(response) as zipfile:
        namelist = zipfile.namelist()
        csv_buf = StringIO(zipfile.read(namelist[0]).decode('utf-8'))
        csv_data = pd.read_csv(csv_buf)

        output_map[(b, n_epochs)] = csv_data
        
    print('Getting coordinate arrays')
    lats = np.unique(csv_data['latitude'].values)
    lons = np.unique(csv_data['longitude'].values)
    times = np.unique(csv_data['time'].values)
    
    data_col = [c for c in csv_data.columns if c not in ['time', 'latitude', 'longitude']][0]
    
    print('Gridding the result')
    
    vals_3d = np.empty((len(times), len(lats), len(lons)), dtype='float32')
    data_dict = {}
    
    for t in csv_data.itertuples(index=False):
        key = (t.time, t.latitude, t.longitude)
        data_dict[key] = getattr(t, data_col)
    
    for i, t in enumerate(times):
        for j, lat in enumerate(lats):
            for k, lon in enumerate(lons):
                vals_3d[i, j, k] = data_dict.get((t, lat, lon), np.nan)
            

    v_min = np.nanmin(vals_3d)
    v_max = np.nanmax(vals_3d)
    
    pcm = ax.pcolormesh(*np.meshgrid(lons, lats), vals_3d[0])
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'{n_epochs} epochs: {b}')
    return [v_min, v_max]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14,12))

minmax = []

minmax.append(kauri('full', 20, axes[0][0]))
minmax.append(kauri('closeup', 20, axes[0][1]))
minmax.append(kauri('full', 200, axes[1][0]))
minmax.append(kauri('closeup', 200, axes[1][1]))

minmax = np.array(minmax)

cb = fig.colorbar(
    ScalarMappable(norm=Normalize(vmin=min(minmax[:, 0]), vmax=max(minmax[:, 1])), cmap='viridis'), 
    label='Canopy height (98th percentile) [m]', 
    ax=axes.ravel().tolist()
)

In [ ]:
for o in output_map:
    output = output_map[o].drop('time', axis=1)
    col = output.columns[-1]
    output['val'] = output[col]
    output.to_csv(f'kauri_{o[0]}_{o[1]}.ply', header=False, index=False)